In [1]:
import pandas as pd
import re

In [26]:
# load initial dataframe
df = pd.read_csv("../data/IKSHPK_page_posts.csv")

In [50]:
# Data extraction

data = {}

for index, row in df.iterrows():
    # Try to find number of cases
    text = str(row["text"])
    paragraph = re.search(r"Rastet pozitive janë nga: (.*)\.", text)
    
    # skip if not
    if paragraph is None:
        continue
    
    print(row['time'])
    cases = paragraph.group()
    
    # Find text for each komune
    multi = re.findall(r"\w+ (?:\w+|\w+\s+\w+) \d+ \w+", cases)
    print(multi)
    
    for komuna in multi:
        # Special cases
        if ' 1 rast' in komuna:
            continue
        elif 'me nga 2 raste' in komuna:
            continue
        elif 'me nga 3 raste' in komuna:
            continue
        elif 'nga 2 raste' in komuna:
            continue
        elif 'sipas vendbanimit' in komuna:
            continue
        elif 'komuna' not in komuna:
            continue
        
        # Find amount of new cases 
        new_cases = re.findall(r'\d+', komuna)
        # Get indivual komunas
        new_komuna_txt = re.findall(r'\w+ (?:\w+|\w+\s+\w+) \d', komuna)
        # Get only komuna name
        new_komuna = new_komuna_txt[0][re.search(r"\b[A-Z]", new_komuna_txt[0]).span()[0]:-2]
        # Add to data, k,v = 'komuna': nr_new_cases
        data[new_komuna] = int(new_cases[0])
    
    # Find 1 case komunas
    single = re.findall(r"dhe(.*)\.", cases)
    single = single[0]
    
    # TODO: Optimize code below.
    
    # Special case Hani i Elezit
    if 'Hani i Elezit' in single:
        data['Hani i Elezit'] = 1
        single = single.replace('Hani i Elezit', '')
    
    if 'Han i Elezit' in single:
        data['Hani i Elezit'] = 1
        single = single.replace('Han i Elezit', '')
    
    # Special case Mitrovicë e Veriut
    if 'Mitrovicë e Veriut' in single:
        data['Mitrovicë e Veriut'] = 1
        single = single.replace('Mitrovicë e Veriut', '')
    
    # Two word komunas:
    two_komuna = re.findall(r'(?:[A-Z]\w+\s[A-Z]\w+)', single)
    for komuna in two_komuna:
        data[komuna] = 1
        single = single.replace(komuna, '')
    
    # Single word Komunas:
    single_komunat = re.findall(r'[A-Z]\w+', single)
    for komuna in single_komunat:
        data[komuna] = 1
    
    print(data)

2020-08-11 14:59:11
['komuna Prishtinë 46 raste', 'komuna Gjilan 24 raste', 'komuna Gjakovë 15 raste', 'komuna Viti 13 raste', 'komuna Fushë Kosovë 9 raste', 'komuna Pejë 7 raste', 'komuna Drenas 6 raste', 'komuna Prizren 6 raste', 'komuna Suharekë 6 raste', 'komuna Ferizaj 5 raste', 'komuna Klinë 5 raste', 'komuna Lipjan 5 raste', 'komuna Dragash 4 raste', 'komuna Istog 3 raste', 'komuna Kamenicë 3 raste', 'komuna Malishevë 3 raste', 'komuna Obiliq 3 raste', 'komuna Podujevë 3 raste', 'komuna Vushtrri 2 raste']
{'Prishtinë': 46, 'Gjilan': 24, 'Gjakovë': 15, 'Viti': 13, 'Fushë Kosovë': 9, 'Pejë': 7, 'Drenas': 6, 'Prizren': 6, 'Suharekë': 6, 'Ferizaj': 5, 'Klinë': 5, 'Lipjan': 5, 'Dragash': 4, 'Istog': 3, 'Kamenicë': 3, 'Malishevë': 3, 'Obiliq': 3, 'Podujevë': 3, 'Vushtrri': 2, 'Mitrovicë': 1, 'Rahovec': 1, 'Shtime': 1}
2020-08-10 15:00:51
['komuna Prishtinë 78 raste', 'komuna Gjilan 19 raste', 'komuna Mitrovicë 15 raste', 'komuna Fushë Kosovë 9 raste', 'komuna Pejë 8 raste', 'komuna Vi

In [31]:
# get the paragraph with all the municipalities
cols = ["komunat", "time"]
df_text = pd.DataFrame(columns=cols)

for index, row in df.iterrows():
    s = str(row["text"])
    start = "Rastet pozitive janë nga: "
    end = "Të nderuar qytetarë!"
    text = s[s.find(start)+len(start):s.rfind(end)]
    komunat = text[text.find("komuna e"):text.find(".", text.find("komuna e"))]
    tmp = [(komunat, row["time"])]
    df_tmp = pd.DataFrame.from_records(tmp, columns=cols)
    df_text = df_text.append(df_tmp, ignore_index = True)

In [32]:
df = df_text.loc[df_text["komunat"]!=""].reset_index(drop=True)

In [45]:
df = pd.read_csv("../data/komunat.csv")

In [46]:
df["komunat"] = df["komunat"].apply(lambda x: x.lower())

In [47]:
whereitworks = df.loc[df["time"] == '2020-06-17 19:52:58'].index[0]
df = df.iloc[:whereitworks]

In [52]:
cols = ["komuna", 'new', 'date']
df_new_cases = pd.DataFrame(columns = cols)

# trying to get the data
# TODO: Handle dhe "me nga 1 rast"
for index, row in df.iterrows():
    date = row['time']
    text = row["komunat"]
    half = text.split(" 1 rast")
    if "(" not in half[0]:
        komunat = half[0].split(",")
        for komuna in komunat[:-1]:
            start = 'komuna e '
            end = ' raste'
            s = komuna
            result = s[s.find(start)+len(start):s.rfind(end)]
            tmp = [(result[:-2], result[-2:], date[:10])]
            df_tmp = pd.DataFrame.from_records(tmp, columns=cols)
            df_new_cases = df_new_cases.append(df_tmp, ignore_index = True)
        

In [61]:
df_new_cases = df_new_cases.dropna()
df_new_cases['date'] = pd.to_datetime(df_new_cases['date'])
df_new_cases['new'] = df_new_cases['new'].astype(int)
df_new_cases.head()
df_new_cases.to_csv('komunat_new_cases.csv', index=False)

In [ ]:
# todo remove 1 row where komuna is emtpy
# change date manually from 05.07 to 04.07